# Set up

In [331]:
%pip install rdflib
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [372]:
from rdflib import Graph, URIRef
import os

data_folder = "Data"
g = Graph()
g.parse("Data/rdf_400_sphn_augmented_hybrid.ttl", format="turtle")
#g.parse("Data/rdf_300_sphn.txt", format="turtle")
#g.parse("Data/rdf_400_sphn.txt", format="turtle")
print(f"Loaded {len(g)} triples.")

Loaded 145196 triples.


# Chat GPT Generation

In [60]:
%pip install openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
from openai import OpenAI
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv(dotenv_path="Data/.env")
client = OpenAI()

# Read prompt from file
with open("Data/prompt.txt", "r") as file:
    prompt_text = file.read()

# Read graph text data
with open("Data/ontology.ttl", "r") as f:
    graph_data = f.read()

# Combine both into a full prompt
full_prompt = f"{prompt_text}\n\n{graph_data}"


# Query OpenAI
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": full_prompt}
    ]
)

generated_response = response.choices[0].message.content

# Save to file
with open("Data/output.txt", "w", encoding="utf-8") as out_file:
    out_file.write(generated_response)

print("\n✅ Response saved to output.txt")


✅ Response saved to output.txt


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="Data/.env")
client = OpenAI()

try:
    models = client.models.list()
    print("✅ API key is valid. Available models:")
    for m in models.data:
        print(m.id)
except Exception as e:
    print("❌ Error:", e)

In [373]:
output_file = "Data/output.txt"

if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as file:
        content = file.read()

    # Replace all instances of X or Y with {X} and {Y}
    updated_content = content.replace("{X}", "X").replace("{Y}", "Y")

    # Save the updated content back to the file
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(updated_content)

    print("✅ Updated content saved to output.txt")
else:
    print("❌ File not found.")

✅ Updated content saved to output.txt


# Moving from txt to yaml

In [414]:
import os

output_file = "Data/queries.txt"

question_query_pairs = []

if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            if line.strip():  # Skip empty lines
                updated_line = " ".join(line.split()[1:])
                question_query_pairs.append(updated_line)
list1 = question_query_pairs[::2]  # Elements at even indices
list2 = question_query_pairs[1::2]  # Elements at odd indices

# Combine the two lists into one list of tuples
combined_list = list(zip(list1, list2))

print(f"Loaded {len(combined_list)} question-query pairs.")

Loaded 17 question-query pairs.


In [415]:
updated_question_query_pairs = []

for question, query in combined_list:
    # Remove the first token from the question
    updated_question = " ".join(question.split()[1:])
    updated_question_query_pairs.append((updated_question, query))

# Print the updated pairs
# for q, a in updated_question_query_pairs[:20]:  # Display the first 10 updated pairs
#     print("Updated Question:", q)
#     print("SPARQL Query:", a)

In [416]:
import yaml

output_yaml = "question_query_templates.yaml"

# Consolidating data
data_to_write = [{"Question": q, "Query": a} for q, a in combined_list]

with open(output_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(data_to_write, file, default_flow_style=False, allow_unicode=True)

print(f"YAML file '{output_yaml}' created successfully.")

YAML file 'question_query_templates.yaml' created successfully.


# Generating qqpairs from the templates

In [423]:
#Loading instances of specific classes based on the questions asked.
diagnosis_code_instances = set()
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis'))):
    for _, code_predicate, code in g.triples((s, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode'), None)):
        for _, code_predicate, code in g.triples((code, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodeValue'), None)):                        
            diagnosis_code_instances.add(str(code))
            # print(diagnosis_code_instances)

In [ ]:
drug_code_instances = set()
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    # For each DrugPrescription, find the associated drug codes
    for _, code_predicate, code in g.triples((s, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug'), None)):
        for _, code_predicate, code in g.triples((code, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode'), None)):
            for _, code_predicate, code in g.triples((code, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodeValue'), None)):
                drug_code_instances.add(str(code))

print(f"Found {len(drug_code_instances)} drug code instances.")
print(drug_code_instances)

Found 1 drug code instances.
{'M01AE01'}


In [380]:
substance_code_instances = set()
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    # For each DrugPrescription, find the associated drug codes
    for _, code_predicate, code in g.triples((s, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug'), None)):
        for _, code_predicate, code in g.triples((code, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient'), None)):
            for _, code_predicate, code in g.triples((code, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode'), None)):
                substance_code_instances.add(str(code))

print(f"Found {len(substance_code_instances)} substance code instances.")
print(substance_code_instances)

Found 1652 substance code instances.
{'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#17478081725', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#703398601', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#68084076201', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#904658304', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#93414664', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#58914017110', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#70860045110', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#43547040610', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#43598044674', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#42023017204', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#42023014625', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#378580793', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#168005546', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#3089331', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#904679761', 'https://www.biomed

In [381]:
severity_code_instances = set()
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/AdverseEvent'))):
    # For each AdverseEvent, find the associated severity codes
    for _, code_predicate, code in g.triples((s, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSeverity'), None)):
        severity_code_instances.add(str(code))

print(f"Found {len(severity_code_instances)} severity code instances.")
print(severity_code_instances)

Found 1 severity code instances.
{'Serious'}


In [382]:
lab_test_dates = set()
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/LabTestEvent'))):
    # For each LabTestEvent, find the associated dates
    for _, date_predicate, date in g.triples((s, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDate'), None)):
        lab_test_dates.add(str(date))
print(f"Found {len(lab_test_dates)} lab test dates.")
print(lab_test_dates)

Found 0 lab test dates.
set()


In [383]:
dates_set = set()
for i in range(10):
    dates_set.add(f"2023-10-{i+1:02d}")

print(f"Threshold set: {dates_set}")

Threshold set: {'2023-10-05', '2023-10-04', '2023-10-01', '2023-10-02', '2023-10-06', '2023-10-07', '2023-10-09', '2023-10-08', '2023-10-03', '2023-10-10'}


In [384]:
drug_value_instances = set()
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    # For each DrugPrescription, find the associated drug codes
    for _, code_predicate, code in g.triples((s, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug'), None)):
        for _, code_predicate, code in g.triples((code, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode'), None)):
            for _,code_predicate, code in g.triples((code, URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasValue'), None)):
                drug_value_instances.add(str(code))

print(f"Found {len(drug_value_instances)} drug code instances.")
print(drug_value_instances)

Found 1 drug code instances.
{'M01AE01'}


In [385]:
threshold_set = set()
for i in range(10):
    threshold_set.add(i/1.0)

print(f"Threshold set: {threshold_set}")

Threshold set: {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0}


In [386]:
age_set = set()
for i in range(15, 80):
    age_set.add(i)
print(f"Age set: {age_set}")

Age set: {15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}


In [417]:
import yaml

# Load data from the YAML file
yaml_file = "question_query_templates.yaml"

with open(yaml_file, mode="r", encoding="utf-8") as file:
    question_query_data_original = yaml.safe_load(file)
len(question_query_data_original)

17

In [418]:
# Add template numbers to the question_query_data_original
for index, entry in enumerate(question_query_data_original):
    entry["TemplateNumber"] = index + 1  # Start numbering from 1

# Print the updated list to verify
# for entry in question_query_data_original:
#     print(entry)

In [420]:
import re
pattern = r"\{[a-zA-Z0-9]+\}"
pattern_pref = r"sphn:[a-zA-Z]+"
prefix = "<https://www.biomedit.ch/rdf/sphn-schema/sphn/"
rdf_prefix = "<http://www.w3.org/1999/02/22-rdf-syntax-ns#"
for query in question_query_data_original:
    half_update_query = query["Query"]
    if(re.search(r"rdf:type", half_update_query) is not None):
        half_update_query = re.sub(r"rdf:type", rdf_prefix + "type>" , half_update_query, count = 1)

    while re.search(pattern_pref, half_update_query):
        match = re.search(pattern_pref, half_update_query).group(0)
        replacement = prefix + match.split(":")[1] + ">"
        # print(f"Replacing {match} with {replacement}")
        half_update_query = half_update_query.replace(match, replacement, 1)
    query["Query"] = half_update_query            
    

In [421]:
import re
import random
generated_qq = []
pattern = r"§[a-zA-Z0-9]+§"
prefix = "<https://www.biomedit.ch/rdf/sphn-schema/sphn/"
# Iterate through the question-query data and replace placeholders with actual instances
# Using counter to account for questions with multiple placeholders basically adding lots of templates with one placeholder less
# Random samplin used to reduce running time
counter = 0
question_query_data = question_query_data_original.copy()
while counter <len(question_query_data):
    entry = question_query_data[counter]
    counter += 1
    question = entry.get("Question")
    query = entry.get("Query")
    if "§Diagnosis" in entry["Question"] and "§Diagnosis" in entry["Query"]:
        sampled_diagnosiscode_instances = random.sample(list(diagnosis_code_instances), 10)
        for diagnosis in sampled_diagnosiscode_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = (str(diagnosis).split('/')[-1]).split('#')[-1]
            query_replace = "<"+diagnosis+">"
            # query_replace = "\"" + q_replace+"\""
            updated_question = re.sub(r"§Diagnosis\d*§", q_replace,question , count =1)
            updated_query = re.sub(r"§Diagnosis\d*§", query_replace, query, count =1)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query, "TemplateNumber": entry.get("TemplateNumber")})
        continue
    if "§Threshold§" in entry["Question"] or "§Threshold§" in entry["Query"]:
        sampled_threshold_set = random.sample(list(threshold_set), 10)
        for threshold in sampled_threshold_set:
            updated_question = re.sub(r"§Threshold\d*§", str(threshold), question, count=1)
            updated_query = re.sub(r"§Threshold\d*§", str(threshold), query, count=1)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query, "TemplateNumber": entry.get("TemplateNumber")})
        continue
    if "§Age" in entry["Question"] or "§Age" in entry["Query"]:
        sampled_age_set = random.sample(list(age_set), 10)
        for age in sampled_age_set:
            updated_question = re.sub(r"§Age\d*§", str(age), question, count=1)
            updated_query = re.sub(r"§Age\d*§", str(age), query, count=1)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query, "TemplateNumber": entry.get("TemplateNumber")})
        continue
    if "§Drug" in entry["Question"] or "§Drug" in entry["Query"]:
        sampled_drug_code_instances = random.sample(list(drug_code_instances), 1)
        for drug in sampled_drug_code_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = (str(drug).split('/')[-1]).split('#')[-1]
            query_replace = "<"+drug+">"
            # query_replace = "\"" + q_replace+"\""
            updated_question = re.sub(r"§Drug\d*§", q_replace, question, count=1)
            updated_query = re.sub(r"§Drug\d*§", query_replace, query, count=1)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query, "TemplateNumber": entry.get("TemplateNumber")})
        continue
    if "§Severity" in entry["Question"] or "§Severity" in entry["Query"]:
        sampled_severity_code_instances = random.sample(list(severity_code_instances), 1)
        for severity in sampled_severity_code_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = (str(severity).split('/')[-1]).split('#')[-1]
            query_replace =  "<"+severity+">"
            updated_question = re.sub(r"§Severity\d*§", q_replace, question, count=1)
            updated_query = re.sub(r"§Severity\d*§", query_replace, query, count=1)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query, "TemplateNumber": entry.get("TemplateNumber")})
        continue
    if "§Substance" in entry["Question"] or "§Substance" in entry["Query"]:
        sampled_substance_code_instances = random.sample(list(substance_code_instances), 1)
        for substance in sampled_substance_code_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = (str(substance).split('/')[-1]).split('#')[-1]
            query_replace = "<"+substance+">"
            updated_question = re.sub(r"§Substance\d*§", q_replace, question, count=1)
            updated_query = re.sub(r"§Substance\d*§", query_replace, query, count=1)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query, "TemplateNumber": entry.get("TemplateNumber")})
        continue
    if "§Date" in entry["Question"] or "§Date" in entry["Query"]:
        sampled_lab_test_dates = random.sample(list(dates_set), 5)
        for date in sampled_lab_test_dates:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = (str(date).split('/')[-1]).split('#')[-1]
            query_replace = "<"+date+">"
            updated_question = re.sub(r"§Date\d*§", q_replace, question, count=1)
            updated_query = re.sub(r"§Date\d*§", query_replace, query, count=1)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            else:
                question_query_data.append({"Question":updated_question, "Query": updated_query, "TemplateNumber": entry.get("TemplateNumber")})
        continue
    generated_qq.append({"Question": question, "Query": query, "Template": question_query_data_original[entry.get("TemplateNumber")-1].get("Query") , "QuestionNumber": len(generated_qq) + 1,
                                     "QuestionTemplate": question_query_data_original[entry.get("TemplateNumber")-1].get("Question")})
            


output_yaml = "generated_question_query_pairs.yaml"
with open(output_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(generated_qq, file, default_flow_style=False, allow_unicode=True)   

In [412]:
# Create a dictionary to store one entry per template type
unique_entries = {}

# Iterate through the generated_qq list
for entry in generated_qq:
    template = entry.get("Template", "Unknown Template")
    if template not in unique_entries:
        unique_entries[template] = entry

# Convert the dictionary values to a list
unique_entries_list = list(unique_entries.values())
example_entries_list = []
print(unique_entries_list[1].get("Template"))
for entry in unique_entries_list:
    example_entries_list.append({"question_example": entry.get("Question"), "query_example": entry.get("Query"), "query_template": entry.get("Template"), "question_template": entry.get("QuestionTemplate")})
output_yaml = "question_query_pairs_examples.yaml"
with open(output_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(example_entries_list, file, default_flow_style=False, allow_unicode=True)

print(f"YAML file '{output_yaml}' created successfully.")
# Print the number of unique entries and the list
print(f"Number of unique entries: {len(unique_entries_list)}")
for item in unique_entries_list:
    print(item)

SELECT ?patient WHERE { ?event <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?patient . ?event <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabResult> ?res . ?res <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasQuantityValue> ?val . FILTER(?val > §Threshold§ ) }
YAML file 'question_query_pairs_examples.yaml' created successfully.
Number of unique entries: 14
{'Question': 'Which patients have been diagnosed with I314?', 'Query': 'SELECT ?patient WHERE { ?patient <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDiagnosis> ?diag . ?diag <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?code . ?code <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodeValue> <https://www.biomedit.ch/rdf/sphn-schema/sphn/icd#I314> . }', 'Template': 'SELECT ?patient WHERE { ?patient <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDiagnosis> ?diag . ?diag <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?code . ?code <https://www.biomedit.ch/rdf/sphn-schema/sphn/h

In [407]:
print(len(question_query_data_original))
print(len(question_query_data))

14
44


In [399]:
import yaml
from collections import defaultdict
# Enumerate the templates in the generated_qq list
for index, entry in enumerate(generated_qq, start=1):
    entry["TemplateNumber"] = index
# Group entries by their template
grouped_entries = defaultdict(list)
for entry in generated_qq:
    template = entry.get("Template", "Unknown Template")
    grouped_entries[template].append({
        "Question": entry["Question"],
        "Query": entry["Query"],
        "QuestionNumber": entry["QuestionNumber"]
    })

# Save the grouped entries to a YAML file
output_yaml = "grouped_question_query_pairs.yaml"
with open(output_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(dict(grouped_entries), file, default_flow_style=False, allow_unicode=True)

print(f"YAML file '{output_yaml}' created successfully.")

YAML file 'grouped_question_query_pairs.yaml' created successfully.


In [394]:
import yaml

# Load data from the grouped_question_query_pairs.yaml file
grouped_qq_pairs_file = "grouped_question_query_pairs.yaml"

with open(grouped_qq_pairs_file, mode="r", encoding="utf-8") as file:
    grouped_qq_pairs_data = yaml.safe_load(file)

# Print the keys to see the structure of the data
print(f"Loaded keys: {list(grouped_qq_pairs_data.keys())}")

Loaded keys: ['?patient WHERE { ?presc <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?patient . ?presc <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?drug . ?drug <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?code . ?code <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodeValue> §Drug§ }', 'SELECT (COUNT(DISTINCT ?patient) AS ?numPatients) WHERE { ?patient <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDiagnosis> ?diag . ?diag <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?code . ?code <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodeValue> §Diagnosis§ . }', 'SELECT ?patient WHERE { ?ae <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://www.biomedit.ch/rdf/sphn-schema/sphn/AdverseEvent> . ?ae <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?patient . ?ae <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSeverity> ?sev . FILTER(?sev IN ("§Severity§")) }', 'SELECT ?patient WHERE { ?event <htt

In [413]:
for entry in generated_qq:
    query = entry.get("Query")
    if "§" in query:
        print(f"Query with placeholder found: {query}")
    